# STT : 오디오 파일 분할 및 LJ음성형식 변환

## 1. Setting

- FFmpeg : 오디오 및 비디오 파일을 처리
- pytube : YouTube 동영상 다운로드
- tiktoken : 틱톡 동영상 다운로드
- openai-whisper : OpenAI의 API를 사용하여 텍스트를 생성

In [1]:
!sudo apt update && sudo apt install ffmpeg
!pip install -q openai-whisper pytube tiktoken

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
f

## Youtuve 영상 Load

pytube 라이브러리를 사용하여 YouTube 동영상의 URL을 사용하여

해당 동영상에서 오디오를 추출하고, 이를 MP3 파일 형식으로 다운로드합니다.

다운로드한 파일 명은 "input.mp3" 이름으로 저장됩니다.

In [2]:
from pytube import YouTube

youtube_vd = YouTube('https://youtu.be/ECbfs81yzxE') # 유투브 경로(10분 이내 짜리로)

# only_audio : True(오디오만 선택) or False(동영상 전체)
youtube_vd.streams.filter(only_audio=True).first().download(
    output_path='.', filename='input.mp3')

'/content/./input.mp3'

## 번역 모델 Load

whisper 라이브러리를 사용하여 텍스트 생성 모델을 load 합니다.

"small"은 사전에 학습된 GPT 모델로 일반적으로 더 적은 매개변수를 가지고 있으며, 작은 데이터셋에서 더 빠르게 사용할 수 있습니다.

30초 이상 오디오 데이터 처리 방법은 아래 방법을 참고합니다.

https://github.com/openai/whisper/discussions/136

In [3]:
import whisper

model = whisper.load_model("small")

로드한 모델을 사용하여 오디오 파일에서 텍스트를 추론합니다.

추론된 텍스트 결과에서 처음 300자를 가져옵니다. result는 딕셔너리 형태로 반환됩니다.

In [4]:
vd_text = model.transcribe("input.mp3")

vd_text["text"][:300]

' 안녕하세요 시청자 여러분 모연사임에 해주는 남자 김윤이 입니다. 오늘은 14.9 패치노트 입니다. 이번 패치는 오른템 패치라고 해도 과언이 아닐 정도로 오른템 관련해서 변경되는 게 많습니다. 일단 20개가 추가되고 2개가 삭제되는데요. 대장장의 장갑에도 체력 300과 치명태 30%가 추가됩니다. 아마 이게 이 리스크 있는 오른템이 생겨서 그런 것 같은데 그렇다고 하더라도 말도 안 되는 숙치라고 저는 생각을 해요. 이제 대장장의 장갑이 골드 증강으로 튀어나오면 무조건 골라야 되는 수준으로 보호가 됐습니다. 그리고 오른템을 주는 조우'

"segments"는 텍스트 또는 음성 데이터의 작은 조각이나 세그먼트를 나타냅니다.

이를 통해 발화자가 변경되거나 주제가 변경되는것을 식별 가능하며 텍스트를 분할하여 분석이 가능합니다.

In [5]:
vd_text["segments"][:3]

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 3.36,
  'text': ' 안녕하세요 시청자 여러분 모연사임에 해주는 남자 김윤이 입니다.',
  'tokens': [50364,
   19289,
   41123,
   4264,
   14707,
   11722,
   11557,
   5727,
   13311,
   1517,
   23281,
   1098,
   35266,
   17376,
   2472,
   97,
   1129,
   37589,
   13,
   50532],
  'temperature': 0.0,
  'avg_logprob': -0.3280164757553412,
  'compression_ratio': 1.5747126436781609,
  'no_speech_prob': 0.03104926459491253},
 {'id': 1,
  'seek': 0,
  'start': 3.36,
  'end': 5.44,
  'text': ' 오늘은 14.9 패치노트 입니다.',
  'tokens': [50532,
   23720,
   3499,
   13,
   24,
   35470,
   31678,
   37524,
   8857,
   37589,
   13,
   50636],
  'temperature': 0.0,
  'avg_logprob': -0.3280164757553412,
  'compression_ratio': 1.5747126436781609,
  'no_speech_prob': 0.03104926459491253},
 {'id': 2,
  'seek': 0,
  'start': 8.72,
  'end': 14.76,
  'text': ' 이번 패치는 오른템 패치라고 해도 과언이 아닐 정도로 오른템 관련해서 변경되는 게 많습니다.',
  'tokens': [50800,
   16299,
   35470,
   47255,
   10258,
   10798,
   9

위의 예식로 세그먼트 목록을 담은 vd_text 변수의 텍스트 길이를 출력합니다.

In [6]:
len(vd_text["text"])

2931

"vd_text" 변수에서 추출된 각 세그먼트에 대해 시작 시간, 종료 시간 및 해당 텍스트를 출력합니다.

In [7]:
for r in vd_text['segments']:
    print(f'[{r["start"]:.2f} --> {r["end"]:.2f}] {r["text"]}')

[0.00 --> 3.36]  안녕하세요 시청자 여러분 모연사임에 해주는 남자 김윤이 입니다.
[3.36 --> 5.44]  오늘은 14.9 패치노트 입니다.
[8.72 --> 14.76]  이번 패치는 오른템 패치라고 해도 과언이 아닐 정도로 오른템 관련해서 변경되는 게 많습니다.
[14.76 --> 17.44]  일단 20개가 추가되고 2개가 삭제되는데요.
[17.44 --> 21.80]  대장장의 장갑에도 체력 300과 치명태 30%가 추가됩니다.
[21.80 --> 25.24]  아마 이게 이 리스크 있는 오른템이 생겨서 그런 것 같은데
[25.24 --> 28.40]  그렇다고 하더라도 말도 안 되는 숙치라고 저는 생각을 해요.
[28.40 --> 34.00]  이제 대장장의 장갑이 골드 증강으로 튀어나오면 무조건 골라야 되는 수준으로 보호가 됐습니다.
[34.00 --> 37.16]  그리고 오른템을 주는 조우자함도 증가하고
[37.16 --> 41.84]  그리고 휴대형 대장간이 언제 먹든 4개 중 1개를 선택하도록 변경됩니다.
[41.84 --> 45.88]  이제는 9시즌 오른 전설 때문에 너플했던거죠.
[45.88 --> 48.00]  시스템부터 차근차근 보겠습니다.
[48.00 --> 55.08]  전에 말했던 4월 6스테이지 데미지 1일 증가 그리고 8, 9랩에서 4코스트 확률 너프가 그대로 들어옵니다.
[55.08 --> 57.56]  그리고 9랩 경험 7, 8 증가까지 들어옵니다.
[57.60 --> 60.92]  근데 이거 말고도 3코스트 챔피언 버프가 좀 섞여 있어서
[60.92 --> 63.40]  어떻게 될지는 게임이 들어가 봐야 알 것 같긴 해요.
[63.40 --> 65.48]  조우자란 변경점이 많습니다.
[65.48 --> 68.72]  예전에 말했던 4코 관련 조우자들이 삭제된다.
[68.72 --> 72.12]  초밥을 뒤집여 없는 조우자들이 좀 확률이 감수된다.
[72.12 --> 75.00]  조금 이질적인 조우자

STT 폴더를 신규 생성합니다.

이후 ffmpeg를 사용하여 오디오 파일을 세그먼트별로 분할하고 WAV 방식으로 변환합니다.

In [8]:
!mkdir STT

mkdir: cannot create directory ‘STT’: File exists


In [9]:
# -y: 덮어쓰기를 허용
# -i input.mp3: -i : 입력 파일을 input.mp3로 지정
# -ss {r["start"]}: 시작 시간을 세그먼트의 r["start"] 시간으로 지정
# -to {r["end"]}: 종료 시간을 세그먼트의 r["end"] 시간으로 지정
# -hide_banner: 이 옵션은 ffmpeg의 배너를 감춘다.
# -loglevel error: 로그 레벨을 에러 메시지만 출력하도록 설정
# STT/audio{i+1}.wav : 파일 이름 지정

for i, r in enumerate(vd_text['segments']):
    !ffmpeg -y -i input.mp3 -ss {r["start"]} -to {r["end"]} -hide_banner -loglevel error STT/audio{i+1}.wav # wave : 오디오 파일 형식

구글 코랩와 드라이브를 연결합니다.

Audio() 함수를 이용해 설정한 파일 경로에서 오디오를 로드합니다.

In [10]:
from IPython.display import Audio

Audio("/content/STT/audio10.wav")

In [11]:
with open("metadata_kimluwin.txt", "w", encoding="utf-8") as f:

    for i, r in enumerate(vd_text['segments']):
        f.write(f"audio{i+1}|{r['text'].strip()}|{r['text'].strip()}\n")

WAV 파일과 메타데이터가 포함된 압축 파일이 Google 드라이브에 저장합니다.

하지만 본 과정에서는 테스트를 진행하여 아래 과정은 별도 진행하지 않습니다.

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Do not excute this shell
!mkdir /content/drive/MyDrive/bbanghyong-tts-dataset
!zip -r bbanghyong.zip /content/STT /content/metadata_kimluwin.txt
!cp -a /content/bbanghyong.zip /content/drive/MyDrive/bbanghyong-tts-dataset

In [13]:
!pip install TTS

https://github.com/coqui-ai/TTS라는 저장소를 복제합니다.

Coqui TTS 프로젝트에 대한 공식 GitHub 저장소로, 오픈소스 텍스트 음성 합성 시스템을 제공합니다.

In [14]:
!git clone https://github.com/coqui-ai/TTS

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32844 (delta 21), reused 26 (delta 17), pack-reused 32804
Receiving objects: 100% (32844/32844), 166.21 MiB | 20.89 MiB/s, done.
Resolving deltas: 100% (23813/23813), done.


!tts 명령으로 TTS(Text-To-Speech) 프로그램을 실행합니다.

"Text for TTS"라는 텍스트를 음성으로 변환하고, 그 결과를 "speech.wav" 파일로 저장합니다.

In [22]:
!tts --text "Hello, World's" --out_path speech2.wav

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

IPython의 Audio 클래스를 사용하여 "speech.wav" 파일을 재생하는 오디오를 표시합니다.

In [23]:
from IPython.display import Audio

Audio("./speech2.wav")

#학습

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_LJSpeech_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
#OUT_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), "run", "training")
OUT_PATH = '/content/run'

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 3  # set here the batch size
GRAD_ACUMM_STEPS = 84  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="ljspeech",
    path="/content/wavs/",
    meta_file_train="/content/metadata.txt",
    language="ko",
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


# Training sentences generations
SPEAKER_REFERENCE = [
    "./tests/data/ljspeech/wavs/LJ001-0002.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language


def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "This cake is great. It's so delicious and moist.",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()

 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 2.37kiB/s]

  3%|▎         | 6.32M/211M [00:00<00:03, 63.2MiB/s]
  7%|▋         | 14.8M/211M [00:00<00:02, 76.0MiB/s]
 11%|█         | 22.9M/211M [00:00<00:02, 78.4MiB/s]
 15%|█▍        | 31.3M/211M [00:00<00:02, 80.3MiB/s]
 19%|█▉        | 40.2M/211M [00:00<00:02, 83.5MiB/s]
 23%|██▎       | 49.2M/211M [00:00<00:01, 85.9MiB/s]
 27%|██▋       | 57.8M/211M [00:00<00:01, 85.1MiB/s]
 32%|███▏      | 66.4M/211M [00:00<00:01, 84.3MiB/s]
 36%|███▌      | 74.9M/211M [00:00<00:01, 84.5MiB/s]
 40%|███▉      | 83.3M/211M [00:01<00:01, 82.9MiB/s]
 44%|████▎     | 91.8M/211M [00:01<00:01, 83.4MiB/s]
 48%|████▊     | 100M/211M [00:01<00:01, 83.5MiB/s] 
 52%|█████▏    | 108M/211M [00:01<00:01, 80.7MiB/s]
 55%|█████▌    | 117M/211M [00:01<00:01, 72.6MiB/s]
 59%|█████▉    | 124M/211M [00:01<00:01, 72.9MiB/s]
 63%|██████▎   | 132M/211M [00:01<00:01, 74.5MiB/s]
 66%|██████▋   | 140M/211M [00:01<00:00, 75.6MiB/s]
 70%

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:02<00:00, 73.1MiB/s]
 27%|██▋       | 99.3k/361k [00:00<00:00, 852kiB/s]
100%|██████████| 361k/361k [00:00<00:00, 597kiB/s] 

  0%|          | 5.68M/1.87G [00:00<00:32, 56.8MiB/s]
  1%|          | 13.7M/1.87G [00:00<00:26, 70.8MiB/s]
  1%|          | 22.6M/1.87G [00:00<00:23, 78.7MiB/s]
  2%|▏         | 31.2M/1.87G [00:00<00:22, 81.9MiB/s]
  2%|▏         | 39.4M/1.87G [00:00<00:25, 71.0MiB/s]
  3%|▎         | 48.0M/1.87G [00:00<00:24, 75.5MiB/s]
  3%|▎         | 55.9M/1.87G [00:00<00:23, 76.6MiB/s]
  3%|▎         | 63.9M/1.87G [00:00<00:23, 77.5MiB/s]
  4%|▍         | 71.7M/1.87G [00:00<00:23, 76.4MiB/s]
  4%|▍         | 80.0M/1.87G [00:01<00:22, 78.3MiB/s]
  5%|▍         | 87.9M/1.87G [00:01<00:23, 77.4MiB/s]
  5%|▌         | 95.9M/1.87G [00:01<00:22, 78.3MiB/s]
  6%|▌         | 104M/1.87G [00:01<00:21, 80.5MiB/s] 
  6%|▌         | 113M/1.87G [00:01<00:21, 81.1MiB/s]
  6%|▋         | 121M/1.87G [00:01<00:21, 81.5MiB/s]
  7%|▋         | 130M/1.87G [00:01<

>> DVAE weights restored from: /content/run/XTTS_v2.0_original_model_files/dvae.pth
 | > Found 112 files in /content/wavs


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception